# Phase 4: Running BEAST

```
Parameters
-------------
save_dir: str  
    Path to directory for saving outputs in.

seeds: list of ints
    Seeds to use when running BEAST.


```


In [ ]:
save_dir=''
seeds=''
sbatch_arg_string=''
beast_arg_string=''
beast_xml_path=''
kernel='beast_pype'

In [ ]:
if [ "$beast_xml_path" = "" ];
then
    beast_xml_path=$save_dir/beast.xml
fi

In [ ]:
# For some reason this still has to be called so that slurm's sbatch to use the beast_pype environment.
source activate $kernel

In [ ]:
if [ -e $beast_xml_path ]
    then
    mkdir $save_dir/beast_outputs
    job_ids_file=$save_dir/beast_outputs/slurm_job_ids.txt
    echo "Seed;JobID;Cluster" >> $job_ids_file
    for seed in "${seeds[@]}"
    do
        job_id=$(sbatch -o $save_dir/beast_outputs/run-with-seed-${seed}.out --parsable\
                        $sbatch_arg_string \
                        --wrap="beast -seed ${seed} -prefix '$save_dir/beast_outputs/run-with-seed-${seed}-' -statefile '$save_dir/beast_outputs/run-with-seed-${seed}'  $beast_arg_string '$beast_xml_path'")
        if [[ ${job_id} == *";"* ]]; then
            echo "${seed};${job_id}" >> $job_ids_file
        else
            echo "${seed};${job_id}; " >> $job_ids_file
        fi
    done
    else
    valid_dirs=$(find $save_dir -maxdepth 1 -type d)
    array_to_remove=($save_dir $save_dir/.ipynb_checkpoints)
    for element in "${array_to_remove[@]}"; do valid_dirs=("${valid_dirs[@]/$element}"); done
    valid_dirs=($valid_dirs)
    number_of_chains=$((${#seeds[@]}/${#valid_dirs[@]}))
    seed_index=0
    for valid_dir in "${valid_dirs[@]}"
        do
            beast_xml_path=$valid_dir/beast.xml
            mkdir $valid_dir/beast_outputs
            job_ids_file=$valid_dir/beast_outputs/slurm_job_ids.txt
            echo "Seed;JobID;Cluster" >> $job_ids_file
            for rep in $(seq 1 $number_of_chains)
            do
                seed=${seeds[$seed_index]}
                job_id=$(sbatch -o $valid_dir/beast_outputs/run-with-seed-${seed}.out --parsable\
                            $sbatch_arg_string \
                            --wrap="beast -seed ${seed} -prefix '$valid_dir/beast_outputs/run-with-seed-${seed}-' -statefile '$valid_dir/beast_outputs/run-with-seed-${seed}'  $beast_arg_string '$beast_xml_path'")
            if [[ ${job_id} == *";"* ]]; then
                echo "${seed};${job_id}" >> $job_ids_file
            else
                echo "${seed};${job_id}; " >> $job_ids_file
            fi
            seed_index=$seed_index+1
            done
        done
fi

It may take several hours (overnight) for these beast runs to finish. You can check on there progress via `squeue --me`.